In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import pandas as pd
pd.set_option('precision', 3)

# Data Visualisation Libraries
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

!pip install seaborn --upgrade
import seaborn as sns
sns.set_style('darkgrid')

# Statistics
from scipy.stats import chi2_contingency
from imblearn.over_sampling import SMOTE

# Machine Learning
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import learning_curve

from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score, recall_score, precision_score, auc, roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 KB 11.4 MB/s eta 0:00:00
  Attempting uninstall: seaborn
    Found existing installation: seaborn 0.11.2
    Uninstalling seaborn-0.11.2:
      Successfully uninstalled seaborn-0.11.2


Test data 

In [2]:
travel_data_test_df = pd.read_csv('/content/drive/MyDrive/MIT/Hackathon/Traveldata_test_(1).csv')
display(travel_data_test_df.head())
display(travel_data_test_df.info())
display(travel_data_test_df.describe())

,ID,Gender,Customer_Type,Age,Type_Travel,Travel_Class,Travel_Distance,Departure_Delay_in_Mins,Arrival_Delay_in_Mins
0,99900001,Female,NaN,36.0,Business Travel,Business,532,0.0,0.0
1,99900002,Female,Disloyal Customer,21.0,Business Travel,Business,1425,9.0,28.0
2,99900003,Male,Loyal Customer,60.0,Business Travel,Business,2832,0.0,0.0
3,99900004,Female,Loyal Customer,29.0,Personal Travel,Eco,1352,0.0,0.0
4,99900005,Male,Disloyal Customer,18.0,Business Travel,Business,1610,17.0,0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35602 entries, 0 to 35601
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       35602 non-null  int64  
 1   Gender                   35572 non-null  object 
 2   Customer_Type            32219 non-null  object 
 3   Age                      35591 non-null  float64
 4   Type_Travel              32154 non-null  object 
 5   Travel_Class             35602 non-null  object 
 6   Travel_Distance          35602 non-null  int64  
 7   Departure_Delay_in_Mins  35573 non-null  float64
 8   Arrival_Delay_in_Mins    35479 non-null  float64
dtypes: float64(3), int64(2), object(4)
memory usage: 2.4+ MB


None

,ID,Age,Travel_Distance,Departure_Delay_in_Mins,Arrival_Delay_in_Mins
count,3.560e+04,35591.000,35602.000,35573.000,35479.000
mean,9.992e+07,39.447,1987.152,14.881,15.309
std,1.028e+04,15.138,1024.309,37.895,38.531
min,9.990e+07,7.000,50.000,0.000,0.000
25%,9.991e+07,27.000,1360.000,0.000,0.000
50%,9.992e+07,40.000,1929.000,0.000,0.000
75%,9.993e+07,51.000,2559.000,13.000,13.000
max,9.994e+07,85.000,6868.000,978.000,970.000


In [3]:
survey_data_test_df = pd.read_csv('/content/drive/MyDrive/MIT/Hackathon/Surveydata_test_(1).csv')
display(survey_data_test_df.head())
display(survey_data_test_df.info())
display(survey_data_test_df.describe())

,ID,Seat_Comfort,Seat_Class,Arrival_Time_Convenient,Catering,Platform_Location,Onboard_Wifi_Service,Onboard_Entertainment,Online_Support,Ease_of_Online_Booking,Onboard_Service,Legroom,Baggage_Handling,CheckIn_Service,Cleanliness,Online_Boarding
0,99900001,Acceptable,Green Car,Acceptable,Acceptable,Manageable,Needs Improvement,Excellent,Good,Excellent,Excellent,Excellent,Excellent,Good,Excellent,Poor
1,99900002,Extremely Poor,Ordinary,Good,Poor,Manageable,Acceptable,Poor,Acceptable,Acceptable,Excellent,Acceptable,Good,Acceptable,Excellent,Acceptable
2,99900003,Excellent,Ordinary,Excellent,Excellent,Very Convenient,Excellent,Excellent,Excellent,Needs Improvement,Needs Improvement,Needs Improvement,Needs Improvement,Good,Needs Improvement,Excellent
3,99900004,Acceptable,Green Car,Excellent,Acceptable,Very Convenient,Poor,Acceptable,Excellent,Poor,Acceptable,Needs Improvement,Excellent,Excellent,Excellent,Poor
4,99900005,Excellent,Ordinary,Extremely Poor,Excellent,Needs Improvement,Excellent,Excellent,Excellent,Excellent,NaN,Acceptable,Excellent,Excellent,Excellent,Excellent


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35602 entries, 0 to 35601
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ID                       35602 non-null  int64 
 1   Seat_Comfort             35580 non-null  object
 2   Seat_Class               35602 non-null  object
 3   Arrival_Time_Convenient  32277 non-null  object
 4   Catering                 32245 non-null  object
 5   Platform_Location        35590 non-null  object
 6   Onboard_Wifi_Service     35590 non-null  object
 7   Onboard_Entertainment    35594 non-null  object
 8   Online_Support           35576 non-null  object
 9   Ease_of_Online_Booking   35584 non-null  object
 10  Onboard_Service          32730 non-null  object
 11  Legroom                  35577 non-null  object
 12  Baggage_Handling         35562 non-null  object
 13  CheckIn_Service          35580 non-null  object
 14  Cleanliness              35600 non-nul

None

,ID
count,3.560e+04
mean,9.992e+07
std,1.028e+04
min,9.990e+07
25%,9.991e+07
50%,9.992e+07
75%,9.993e+07
max,9.994e+07


In [4]:
test_df_list = [survey_data_test_df, travel_data_test_df]
test_merged_df = pd.merge(travel_data_test_df, survey_data_test_df, how='inner', on=['ID'])

pd.set_option('display.max_columns', None)

display(test_merged_df.head())
display(test_merged_df.info())

,ID,Gender,Customer_Type,Age,Type_Travel,Travel_Class,Travel_Distance,Departure_Delay_in_Mins,Arrival_Delay_in_Mins,Seat_Comfort,Seat_Class,Arrival_Time_Convenient,Catering,Platform_Location,Onboard_Wifi_Service,Onboard_Entertainment,Online_Support,Ease_of_Online_Booking,Onboard_Service,Legroom,Baggage_Handling,CheckIn_Service,Cleanliness,Online_Boarding
0,99900001,Female,NaN,36.0,Business Travel,Business,532,0.0,0.0,Acceptable,Green Car,Acceptable,Acceptable,Manageable,Needs Improvement,Excellent,Good,Excellent,Excellent,Excellent,Excellent,Good,Excellent,Poor
1,99900002,Female,Disloyal Customer,21.0,Business Travel,Business,1425,9.0,28.0,Extremely Poor,Ordinary,Good,Poor,Manageable,Acceptable,Poor,Acceptable,Acceptable,Excellent,Acceptable,Good,Acceptable,Excellent,Acceptable
2,99900003,Male,Loyal Customer,60.0,Business Travel,Business,2832,0.0,0.0,Excellent,Ordinary,Excellent,Excellent,Very Convenient,Excellent,Excellent,Excellent,Needs Improvement,Needs Improvement,Needs Improvement,Needs Improvement,Good,Needs Improvement,Excellent
3,99900004,Female,Loyal Customer,29.0,Personal Travel,Eco,1352,0.0,0.0,Acceptable,Green Car,Excellent,Acceptable,Very Convenient,Poor,Acceptable,Excellent,Poor,Acceptable,Needs Improvement,Excellent,Excellent,Excellent,Poor
4,99900005,Male,Disloyal Customer,18.0,Business Travel,Business,1610,17.0,0.0,Excellent,Ordinary,Extremely Poor,Excellent,Needs Improvement,Excellent,Excellent,Excellent,Excellent,NaN,Acceptable,Excellent,Excellent,Excellent,Excellent


<class 'pandas.core.frame.DataFrame'>
Int64Index: 35602 entries, 0 to 35601
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       35602 non-null  int64  
 1   Gender                   35572 non-null  object 
 2   Customer_Type            32219 non-null  object 
 3   Age                      35591 non-null  float64
 4   Type_Travel              32154 non-null  object 
 5   Travel_Class             35602 non-null  object 
 6   Travel_Distance          35602 non-null  int64  
 7   Departure_Delay_in_Mins  35573 non-null  float64
 8   Arrival_Delay_in_Mins    35479 non-null  float64
 9   Seat_Comfort             35580 non-null  object 
 10  Seat_Class               35602 non-null  object 
 11  Arrival_Time_Convenient  32277 non-null  object 
 12  Catering                 32245 non-null  object 
 13  Platform_Location        35590 non-null  object 
 14  Onboard_Wifi_Service  

None

In [5]:
test_df = test_merged_df.copy()

In [6]:
test_df = test_df.drop(['Type_Travel', "Seat_Class", "Arrival_Time_Convenient", "Departure_Delay_in_Mins"], axis=1)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35602 entries, 0 to 35601
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      35602 non-null  int64  
 1   Gender                  35572 non-null  object 
 2   Customer_Type           32219 non-null  object 
 3   Age                     35591 non-null  float64
 4   Travel_Class            35602 non-null  object 
 5   Travel_Distance         35602 non-null  int64  
 6   Arrival_Delay_in_Mins   35479 non-null  float64
 7   Seat_Comfort            35580 non-null  object 
 8   Catering                32245 non-null  object 
 9   Platform_Location       35590 non-null  object 
 10  Onboard_Wifi_Service    35590 non-null  object 
 11  Onboard_Entertainment   35594 non-null  object 
 12  Online_Support          35576 non-null  object 
 13  Ease_of_Online_Booking  35584 non-null  object 
 14  Onboard_Service         32730 non-null

In [7]:
test_df['Gender'] = LabelEncoder().fit_transform(test_df['Gender'])

test_df['Ease_of_Online_Booking'] = test_df['Ease_of_Online_Booking'].map({
    'Extremely Poor': 0,
    'Poor': 1,
    'Needs Improvement': 2,
    'Acceptable' : 3,
    'Good' : 4,
    'Excellent' : 5
    })

test_df['Onboard_Entertainment'] = test_df['Onboard_Entertainment'].map({
    'Extremely Poor': 0,
    'Poor': 1,
    'Needs Improvement': 2,
    'Acceptable' : 3,
    'Good' : 4,
    'Excellent' : 5
})

test_df['Online_Support'] = test_df['Online_Support'].map({
    'Extremely Poor': 0,
    'Poor': 1,
    'Needs Improvement': 2,
    'Acceptable' : 3,
    'Good' : 4,
    'Excellent' : 5
})

test_df['Onboard_Wifi_Service'] = test_df['Onboard_Wifi_Service'].map({
    'Extremely Poor': 0,
    'Poor': 1,
    'Needs Improvement': 2,
    'Acceptable' : 3,
    'Good' : 4,
    'Excellent' : 5
})


test_df['Seat_Comfort'] = test_df['Seat_Comfort'].map({
    'Extremely Poor': 0,
    'Poor': 1,
    'Needs Improvement': 2,
    'Acceptable' : 3,
    'Good' : 4,
    'Excellent' : 5
})

test_df['Baggage_Handling'] = test_df['Baggage_Handling'].map({
    'Extremely Poor': 0,
    'Poor': 1,
    'Needs Improvement': 2,
    'Acceptable' : 3,
    'Good' : 4,
    'Excellent' : 5
})

test_df['CheckIn_Service'] = test_df['CheckIn_Service'].map({
    'Extremely Poor': 0,
    'Poor': 1,
    'Needs Improvement': 2,
    'Acceptable' : 3,
    'Good' : 4,
    'Excellent' : 5
})


test_df['Customer_Type'] = test_df['Customer_Type'].map({
    'Loyal Customer': 0,
    'Disloyal Customer': 1
})

test_df['Cleanliness'] = test_df['Cleanliness'].map({
    'Extremely Poor': 0,
    'Poor': 1,
    'Needs Improvement': 2,
    'Acceptable' : 3,
    'Good' : 4,
    'Excellent' : 5
})


test_df['Online_Boarding'] = test_df['Online_Boarding'].map({
    'Extremely Poor': 0,
    'Poor': 1,
    'Needs Improvement': 2,
    'Acceptable' : 3,
    'Good' : 4,
    'Excellent' : 5
})


test_df['Travel_Class'] = test_df['Travel_Class'].map({
    'Business': 0,
    'Eco': 1
})


test_df['Catering'] = test_df['Catering'].map({
    'Extremely Poor': 0,
    'Poor': 1,
    'Needs Improvement': 2,
    'Acceptable' : 3,
    'Good' : 4,
    'Excellent' : 5
})

test_df['Onboard_Service'] = test_df['Onboard_Service'].map({
    'Extremely Poor': 0,
    'Poor': 1,
    'Needs Improvement': 2,
    'Acceptable' : 3,
    'Good' : 4,
    'Excellent' : 5
})

test_df['Legroom'] = test_df['Legroom'].map({
    'Extremely Poor': 0,
    'Poor': 1,
    'Needs Improvement': 2,
    'Acceptable' : 3,
    'Good' : 4,
    'Excellent' : 5
})

test_df['Platform_Location'] = test_df['Platform_Location'].map({
    'Very Inconvenient': 0,
    'Inconvenient': 1,
    'Needs Improvement': 2,
    'Manageable' : 3,
    'Convenient' : 4,
    'Very Convenient' : 5
})

In [8]:
all_columns = test_df.columns.values.tolist()
continuous = ['Age', 'Departure_Delay_in_Mins', 'Arrival_Delay_in_Mins', 'Travel_Distance']
categorical = list(set(all_columns) - set(continuous) - set(["ID", "Overall_Experience"]))

#categorical = ['Gender', 'Seat_Comfort', 'Arrival_Time_Convenient', 'Baggage_Handling', 'Cleanliness', 'Legroom']

print('Continuous: ', ', '.join(continuous))
print('Categorical: ', ', '.join(categorical))

Continuous:  Age, Departure_Delay_in_Mins, Arrival_Delay_in_Mins, Travel_Distance
Categorical:  Travel_Class, Seat_Comfort, Baggage_Handling, CheckIn_Service, Onboard_Wifi_Service, Legroom, Gender, Online_Support, Catering, Online_Boarding, Platform_Location, Onboard_Entertainment, Customer_Type, Ease_of_Online_Booking, Cleanliness, Onboard_Service


In [9]:
display(test_df.isna().sum())

all_cols = test_df.columns

for item in all_cols:
  mean_value=test_df[item].mean()
  test_df[item].fillna(value=mean_value, inplace=True)
  
display(test_df.isna().sum())

display(test_df.info())

display(test_df.head())


ID                           0
Gender                       0
Customer_Type             3383
Age                         11
Travel_Class                 0
Travel_Distance              0
Arrival_Delay_in_Mins      123
Seat_Comfort                22
Catering                  3357
Platform_Location           12
Onboard_Wifi_Service        12
Onboard_Entertainment        8
Online_Support              26
Ease_of_Online_Booking      18
Onboard_Service           2872
Legroom                     25
Baggage_Handling            40
CheckIn_Service             22
Cleanliness                  2
Online_Boarding              2
dtype: int64

ID                        0
Gender                    0
Customer_Type             0
Age                       0
Travel_Class              0
Travel_Distance           0
Arrival_Delay_in_Mins     0
Seat_Comfort              0
Catering                  0
Platform_Location         0
Onboard_Wifi_Service      0
Onboard_Entertainment     0
Online_Support            0
Ease_of_Online_Booking    0
Onboard_Service           0
Legroom                   0
Baggage_Handling          0
CheckIn_Service           0
Cleanliness               0
Online_Boarding           0
dtype: int64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35602 entries, 0 to 35601
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      35602 non-null  int64  
 1   Gender                  35602 non-null  int64  
 2   Customer_Type           35602 non-null  float64
 3   Age                     35602 non-null  float64
 4   Travel_Class            35602 non-null  int64  
 5   Travel_Distance         35602 non-null  int64  
 6   Arrival_Delay_in_Mins   35602 non-null  float64
 7   Seat_Comfort            35602 non-null  float64
 8   Catering                35602 non-null  float64
 9   Platform_Location       35602 non-null  float64
 10  Onboard_Wifi_Service    35602 non-null  float64
 11  Onboard_Entertainment   35602 non-null  float64
 12  Online_Support          35602 non-null  float64
 13  Ease_of_Online_Booking  35602 non-null  float64
 14  Onboard_Service         35602 non-null

None

,ID,Gender,Customer_Type,Age,Travel_Class,Travel_Distance,Arrival_Delay_in_Mins,Seat_Comfort,Catering,Platform_Location,Onboard_Wifi_Service,Onboard_Entertainment,Online_Support,Ease_of_Online_Booking,Onboard_Service,Legroom,Baggage_Handling,CheckIn_Service,Cleanliness,Online_Boarding
0,99900001,0,0.182,36.0,0,532,0.0,3.0,3.0,3.0,2.0,5.0,4.0,5.0,5.000,5.0,5.0,4.0,5.0,1.0
1,99900002,0,1.000,21.0,0,1425,28.0,0.0,1.0,3.0,3.0,1.0,3.0,3.0,5.000,3.0,4.0,3.0,5.0,3.0
2,99900003,1,0.000,60.0,0,2832,0.0,5.0,5.0,5.0,5.0,5.0,5.0,2.0,2.000,2.0,2.0,4.0,2.0,5.0
3,99900004,0,0.000,29.0,1,1352,0.0,3.0,3.0,5.0,1.0,3.0,5.0,1.0,3.000,2.0,5.0,5.0,5.0,1.0
4,99900005,1,1.000,18.0,0,1610,0.0,5.0,5.0,2.0,5.0,5.0,5.0,5.0,3.455,3.0,5.0,5.0,5.0,5.0


In [10]:
test_df = test_df.drop(['ID'], axis=1)

In [11]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35602 entries, 0 to 35601
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Gender                  35602 non-null  int64  
 1   Customer_Type           35602 non-null  float64
 2   Age                     35602 non-null  float64
 3   Travel_Class            35602 non-null  int64  
 4   Travel_Distance         35602 non-null  int64  
 5   Arrival_Delay_in_Mins   35602 non-null  float64
 6   Seat_Comfort            35602 non-null  float64
 7   Catering                35602 non-null  float64
 8   Platform_Location       35602 non-null  float64
 9   Onboard_Wifi_Service    35602 non-null  float64
 10  Onboard_Entertainment   35602 non-null  float64
 11  Online_Support          35602 non-null  float64
 12  Ease_of_Online_Booking  35602 non-null  float64
 13  Onboard_Service         35602 non-null  float64
 14  Legroom                 35602 non-null

In [12]:
test_df.to_csv("/content/drive/MyDrive/MIT/Hackathon/MergedTestData_NanMean.csv", index=False)